## 准备数据

In [20]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [21]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [22]:
(x, y), (x_test, y_test) = mnist_dataset()
print(x.shape)
W1 = tf.compat.v1.get_variable(shape=(784, 1000), name='W1')
print(W1)

(60000, 28, 28)
<tf.Variable 'W1:0' shape=(784, 1000) dtype=float32, numpy=
array([[-0.05548986,  0.0535214 , -0.00169814, ...,  0.00753841,
         0.02009938, -0.04883188],
       [ 0.0176485 ,  0.00353666, -0.00088965, ..., -0.0524257 ,
         0.0347659 ,  0.05168242],
       [-0.00562605,  0.05685939, -0.00731237, ...,  0.01945076,
         0.00609486, -0.04025821],
       ...,
       [ 0.0319755 ,  0.0302523 , -0.03683082, ..., -0.01378015,
         0.01328975,  0.0066792 ],
       [ 0.05404182, -0.02128858,  0.05237038, ..., -0.03639896,
        -0.01706782,  0.03724303],
       [-0.02343996,  0.02638258, -0.02924831, ..., -0.04192679,
        -0.03086094,  0.05391714]], dtype=float32)>


In [23]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1=tf.compat.v1.get_variable(shape=(784,1000),name="W1",initializer=tf.compat.v1.random_normal_initializer)
        self.b1=tf.compat.v1.get_variable(shape=(1,1000),name="b1",initializer=tf.compat.v1.random_normal_initializer)
        self.W2=tf.compat.v1.get_variable(shape=(1000,666),name="W2",initializer=tf.compat.v1.random_normal_initializer)
        self.b2=tf.compat.v1.get_variable(shape=(1,666),name="b2",initializer=tf.compat.v1.random_normal_initializer)
    
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x=tf.reshape(x,(-1,784))
        hidden = tf.nn.relu(tf.matmul(x,self.W1)+self.b1)
        logits = tf.nn.relu(tf.matmul(hidden,self.W2)+self.b2)
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [24]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [26]:
train_data, test_data = mnist_dataset()
for epoch in range(500):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 94.984535 ; accuracy 0.45751667
epoch 1 : loss 93.95316 ; accuracy 0.46193334
epoch 2 : loss 92.9443 ; accuracy 0.46648332
epoch 3 : loss 91.95789 ; accuracy 0.4711
epoch 4 : loss 90.98973 ; accuracy 0.47515
epoch 5 : loss 90.03605 ; accuracy 0.47956666
epoch 6 : loss 89.093 ; accuracy 0.48335
epoch 7 : loss 88.15199 ; accuracy 0.48678333
epoch 8 : loss 87.2116 ; accuracy 0.49061668
epoch 9 : loss 86.26929 ; accuracy 0.4943
epoch 10 : loss 85.322174 ; accuracy 0.49751666
epoch 11 : loss 84.36929 ; accuracy 0.50115
epoch 12 : loss 83.402115 ; accuracy 0.50371665
epoch 13 : loss 82.421585 ; accuracy 0.5069
epoch 14 : loss 81.42164 ; accuracy 0.50995
epoch 15 : loss 80.398865 ; accuracy 0.5129667
epoch 16 : loss 79.3523 ; accuracy 0.51596665
epoch 17 : loss 78.26984 ; accuracy 0.51926666
epoch 18 : loss 77.15791 ; accuracy 0.52235
epoch 19 : loss 76.00071 ; accuracy 0.5258833
epoch 20 : loss 74.78045 ; accuracy 0.5291
epoch 21 : loss 73.4809 ; accuracy 0.5323333
epoch 22 : 